In [4]:
!pip install -qU \
langchain==0.1.1 \
langchain-community==0.0.13 \
openai==0.27.7 \
tiktoken==0.4.0 \
pinecone-client==3.1.0 \
pinecone-datasets==0.7.0 \
pinecone-notebooks \
pypdf


In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [6]:
!mkdir pdfs

mkdir: pdfs: File exists


In [7]:
loader=PyPDFDirectoryLoader("pdfs")
data=loader.load()
print(data[0])

page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recur

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)
print(text_chunks)

In [9]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [10]:
len(text_chunks)

91

In [11]:
print(text_chunks[-1].page_content)

just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention
at layer 5 of 6. The heads clearly learned to perform different tasks.
15


In [12]:
import os
os.environ["OPENAI_API_KEY"]=""

In [13]:
embedding=OpenAIEmbeddings()
len(embedding.embed_query("How are you"))

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


1536

In [14]:
len(embedding.embed_query("hi i am fine"))

1536

In [ ]:
embedding.embed_query("hi i am fine")

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
#from pinecone import Pinecone as PC, ServerlessSpec
print(PINECONE_API_KEY)

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
print(PINECONE_API_KEY)
# configure client
pc = Pinecone(api_key=PINECONE_API_KEY)

In [53]:
index_name="quickstart"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print('Deleted:'+index_name)

Deleted:quickstart


In [54]:
import time
pc.create_index(
    name="quickstart",
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [55]:
print(pc.list_indexes().names())

['quickstart']


In [56]:
index=pc.Index("quickstart")
time.sleep(1)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
# Function to generate embeddings
import openai
def generate_embeddings(text_chunks):
    embeddings = []
    for chunk in text_chunks:
        response = openai.Embedding.create(
            input=chunk.page_content,
            model="text-embedding-ada-002"
        )
        embeddings.append(response['data'][0]['embedding'])
        print(response['data'][0]['embedding'])
    return embeddings
embeddings=generate_embeddings(text_chunks)

In [59]:
# Function to upsert into Pinecone
def upsert_embeddings(index, text_chunks, embeddings):
    ids = [f'chunk_{i}' for i in range(len(text_chunks))]
    metadata = [{'text': chunk.page_content,'source':chunk_id} for chunk,chunk_id in zip(text_chunks,ids)]

    index.upsert(vectors=zip(ids, embeddings, metadata))
upsert_embeddings(index, text_chunks, embeddings)
time.sleep(5)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 91}},
 'total_vector_count': 91}

In [61]:
from langchain.embeddings.openai import OpenAIEmbeddings

# get openai api key from platform.openai.com
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [62]:
query = "Who wrote this paper?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'chunk_0'}),
 Document(page_content='[37] Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton. Grammar as a foreign language. In\nAdvances in Neural Information Processing Systems , 2015.\n[38] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang\nMacherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine\ntranslation system: Bridging the gap between human and machine translation. arXiv preprint\narXiv:1609.0

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
qa.run(query)

'The paper "Attention Is All You Need" was written by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, and Aidan N. Gomez.'

In [65]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
qa_with_sources(query)

{'question': 'Who wrote this paper?',
 'answer': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, and Aidan N. Gomez wrote this paper.\n',
 'sources': 'chunk_0'}

In [66]:
qa_with_sources("what are transformers,why attention is important")

{'question': 'what are transformers,why attention is important',
 'answer': 'Transformers are a model architecture that relies entirely on an attention mechanism to draw global dependencies between input and output sequences. Attention is important because it allows for significantly more parallelization and faster training compared to architectures based on recurrent or convolutional layers. \n',
 'sources': 'chunk_10, chunk_67, chunk_14, chunk_15'}

In [67]:
pc.delete_index(index_name)